# 8. Generative Deep Learning

### 8.5 - Introduction to Generative Adversarial Networks (GANs)

GANs were proposed in 2014 by Ian Goodfellow et al. and are an alternative to VAEs for learning latent spaces of images. They enable the generation of fairly realistic synthetic images by forcing the generated images to be statistically almost indistinguishable from real ones.

An intuitive way to understand GANs is to imagine a forger trying to create a fake Picasso painting. At first, the forger is very bad - he tries to pass of his fake painting as a real one to an art dealer and the art dealer immediately spots the fake painting. The art dealer then gives the forger feedback on what makes a Picasso look like a Picasso. The forger goes back to the studio to create new fakes. As time goes on, the forger is better at imitating the style of Picasso, and the art dealer becomes better at spotting fakes. In the end, they both have excellent fake Picassos.

The GAN is like so - it has a <b>generator network</b> acting as the forger and the <b>discriminator network</b> acting as the art dealer. The generator network takes as input a random vector and decodes it to a synthetic image, while the discriminator takes as input a candidate image and predicts whether the image is from the training set (authentic) or the generator network.

<img src="img91.png" width="600">

A GAN is a system where the optimisation minimum is not fixed. With GAN, every step taken down the hill changes the entire landscape a little. It is a dynamic system where the optimization process is seeking not a minimum, but an equilibrium between two forces. Hence, GANs are notoriously difficult to train - getting a GAN to work requires lots of careful tuning of the model architecture and training parameters.

Here, we will look at deep convolutional GAN (DCGAN): a GAN where the generator and discriminator are deep convnets. The GAN will be trained on images from CIFAR10.

1. A generator network mapes vecotrs of shape `(latent_dim,)` to images of shape `(32,32,3)`.
2. A discriminator network maps images of shape `(32,32,3)` to a binary score estimating the probability if the image is real.
3. A `gan` network chains the genrator together. Thus, the `gan` network maps latent space vectors to the discriminator's assessment of the realism of these latent vectors as decoded by the generator.
4. You train the discriminator using examples of real and fake images using real & fake labels, just like any other classification model.
5. To train the generator, you use gradients of the generator's weights with regard to the loss of the `gan` model. This means, at every step you move the weights of the generator in a direction that makes the discriminator more likely to classify as "real" the images decoded by the generator. In other words, you train the generator to "fool" the discriminator.

Some of the tricks used to implement a GAN include:

- Using `tanh` instead of `sigmoid` as the final activation for the generator
- Sample points from the latent space using a Gaussian distribution
- Add stochasticity to induce robustness

When training, the steps are:

1. Draw random points from the latent space
2. Generate images using the `generator` with the random points in (1)
3. Mix the generated images with real ones
4. Train the `discriminator` using these mixed images with corresponding targets: either "real" for real images or "fake" for generated images
5. Draw new random points in the latent space
6. Train `gan` using these random vectors, with targets that all say "these are real images". These updates the weights of the generator to move them toward getting the discriminator to predict "these are real images" for generated images: this trains the `generator` to fool the `discriminator`.

<hr>

From HandsML:

The fundamental idea behind GANs are simple - make neural networks compete each other in the hope that this competition will push them to excel. A GAN is composed into two networks:

<b>Generator</b> - Takes a random distribution as input and outputs a generated sample (like an image). The random inputs is the latent representations of the sample to be generated. In this case, the generator has the same functionality as the decoder in a VAE, and it can be used in the same way to generate new images. However, it is trained very differently.

<b>Discriminator</b> - Takes either a fake image from the generator or a real image from the training set as the input, and guesses whether the input image is fake or real.

<img src="img85_1.png" width="500">

During training, the generator and discriminator have opposing goals - the discriminator tries discriminate between real and fake images, while the generator tries to produce images to trick the discriminator. Training the GAN has two phases:

1. When training the discriminator, we give a batch of real images and generated images from the generator, with correct labels for "real" and "fake". The discriminator is trained on this label batch for one step, using binary cross-entropy loss. Backpropogation only optimises the weights fof this phase.

2. When training the generator, we first use it to produce a batch of fake images, then ask the discriminator to predict if the images are real. But this time, all the images fed to the discriminator have the label "real". In other words, we want the generator to produce images that the discriminator will (wrongly) believe to be real. The weights of the discriminator are frozen and so backpropogation only affects the weights of the generator.

In the applied example, consider the following:
    
We want a GAN to generate realistic "T-shirt" images. So in the first phase we train the discriminator to distinguish between fake t-shirt images from the generator (which is usually garbage), set $S_{\text{f,1}}$, combine them with actual images $S_{\text{r,1}}$, use the correct labels and train the discriminator. The discriminator learns the weights to distinguish between the fake and real. Pretty straightforward here.

In the second phase, we train the whole GAN. we generate fake images $S_{\text{f,2}}$, label them all real and send them through the discriminator. Now, since the discriminator has learnt what a real "T-shirt" image is, it will likely label many fakes and the <u>loss is high</u>. We now update the weights of only the generator to minimise this loss - generate images that are better the next iteration.

In the third phase, we again generate using the (better) generator and now generate fake images $S_{\text{f,3}}$, combine them with real images $S_{\text{r,3}}$ and send them to the discriminator to minimise loss again. The discriminator is now better to differentiate real from fake images w.r.t. the generator, which has learnt in the second phase to generate images more like the real image. $S_{\text{f,3}}$ is a better forged than  $S_{\text{f,2}}$.

In the fourth phase, we now train the entier GAN. It generates  $S_{\text{f,4}}$, labels them all "real" and sends them to the discriminator. The loss will lead to updating weights of the generator.